# 0. Install dependencies

## Install libraries

In [ ]:
!pip install -q transformers
!pip install anvil-uplink
!pip install google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=d8841e7deec7009191b5fc59206c639e7f108e4cdbfd047c94f82ab57716cef0
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=4c52013e98e15159f28aa1d26a3ea42f59ba53bf8a4625ebee676a778f285693
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


## Import libraries

In [ ]:
import nltk
from transformers import pipeline
from bs4 import BeautifulSoup
from lxml import etree
from lxml import html
import requests
nltk.download('punkt')
nltk.download('stopwords')
import anvil.server
from serpapi import GoogleSearch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Load Models

In [ ]:
summarizer = pipeline("summarization")
classifier_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


## Connect with anvil

In [ ]:
anvil.server.connect("server_6ZWHXGJ4PQ4JERE5LNXKRU7S-65TQBIP2JPT4EMW7")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


# 1. Web scraping

## Scrap URLs from bing's search result

In [ ]:
@anvil.server.callable
def find_websites(bike_name):
  params = {
  "q": f"{bike_name}",
  "location": "Austin, Texas, United States",
  "hl": "en",
  "gl": "us",
  "google_domain": "google.com",
  "api_key": "36ac91e3ea867a4b6820c20c38c8826da2be31cf1cec0e607d6a9fc1dd60a9bc"
  }
  search = GoogleSearch(params)
  results = search.get_dict()
  new_dict = results["organic_results"]
  websites = []
  for i in range(0,len(new_dict)):
    websites.append(new_dict[i]['link'])
  return websites

## Scraping reviews from bikedekho.com

### prerequisite for acces bikedekho.com

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "TE": "Trailers"
}

cookies = {
    "cookie_name": "https://www.bikedekho.com"
}

In [ ]:
@anvil.server.callable
def webScraping_bikedekho(link):
  link += '/reviews'
  try:
    reviews_link = requests.get(link, headers=headers, cookies=cookies)
    bikedekho = BeautifulSoup(reviews_link.text, "html.parser")
    tree = etree.HTML(str(bikedekho))
    result = tree.xpath('//a/span[2]/text()')
    rev_count = int(result[0].split()[0])
    if rev_count % 30 == 0:
      page_count = rev_count / 30
    else:
      page_count = rev_count // 30 + 1
    reviews = []
    for i in range(1,page_count+1):
      specific_review_page = link + f'?pageno={i}/'
      specific_review_page_request = requests.get(specific_review_page,headers=headers, cookies=cookies)
      specific_review_page_html = BeautifulSoup(specific_review_page_request.text,'html.parser')
      tree = etree.HTML(str(specific_review_page_html))
      reviews.append(tree.xpath('//h3/a/text()'))
    user_reviews = []
    for review_list in reviews:
      for review in review_list:
        user_reviews.append(review)
    return user_reviews
  except:
    pass

## Scraping reviews from bikewale.com

In [ ]:
@anvil.server.callable
def webScraping_bikewale(link):
  url = link + '/reviews'
  try:
    home_page_request = requests.get(url)
    home_page_html = BeautifulSoup(home_page_request.content, 'html.parser')
    no_of_review =int(home_page_html.find(class_='review-left-divider').text.split()[0])
    reviews = []
    if no_of_review % 10 == 0:
      no_of_pages = no_of_review / 10
    else:
      no_of_pages = no_of_review // 10 +1
    for i in range(1,no_of_pages+1):
      specific_review_page = url + f'/page/{i}/'
      specific_review_page_request = requests.get(specific_review_page)
      specific_review_page_html = BeautifulSoup(specific_review_page_request.text,'html.parser')
      tree = etree.HTML(str(specific_review_page_html))
      reviews.append(tree.xpath("//div[contains(@class, 'margin-bottom10')]/a/text()"))
    print(reviews)
    user_reviews = []
    for review_list in reviews:
      for review in review_list:
        user_reviews.append(review)
    return user_reviews
  except:
    pass

# 2. Classify reviews

In [ ]:
@anvil.server.callable
def classifier(all_reviews):
  positive = []
  negative = []
  sentiment = [positive,negative]
  for i in range(len(all_reviews)):
    result = classifier_model(all_reviews[i])
    if result[0]['label'] == 'POS':
      positive.append(all_reviews[i])
    elif result[0]['label'] == 'NEG':
      negative.append(all_reviews[i])
  return positive,negative

# 3. Summary of reviews

In [ ]:
@anvil.server.callable
def abstractive_summary(user_input):
  max_chunk = 500
  user_input = user_input.replace('.', '.<eos>')
  user_input = user_input.replace('?', '?<eos>')
  user_input = user_input.replace('!', '!<eos>')
  sentences = user_input.split('<eos>')
  words = len(user_input.split(' '))
  current_chunk = 0 
  chunks = []

  for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        chunks.append(sentence.split(' '))

  for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])
  res = summarizer(chunks, do_sample=False)
  text = ' '.join([summ['summary_text'] for summ in res])
  return text